In [1]:
# Install CatBoost in Colab (if not already available)
!pip install -q catboost

# Import libraries
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings

# Create dataset
X, y = make_classification(
    n_samples=500,
    n_features=10,
    n_informative=5,
    n_redundant=0,
    random_state=2
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Weak learner for AdaBoost
base_learner = DecisionTreeClassifier(max_depth=1, random_state=42)

# AdaBoost (handle sklearn version compatibility)
try:
    ada = AdaBoostClassifier(
        estimator=base_learner,
        n_estimators=100,
        learning_rate=1.0,
        random_state=42
    )
except TypeError:
    ada = AdaBoostClassifier(
        base_estimator=base_learner,
        n_estimators=100,
        learning_rate=1.0,
        random_state=42
    )

ada.fit(X_train, y_train)
ada_acc = accuracy_score(y_test, ada.predict(X_test))

# Try CatBoost; fallback to GradientBoosting if not available
try:
    from catboost import CatBoostClassifier
    cat = CatBoostClassifier(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        verbose=0,
        random_state=42
    )
    cat.fit(X_train, y_train)
    cat_acc = accuracy_score(y_test, cat.predict(X_test))

    print("✅ AdaBoost accuracy:        ", ada_acc)
    print("✅ CatBoost accuracy:        ", cat_acc)

except Exception as e:
    warnings.warn("CatBoost not available. Using GradientBoostingClassifier as fallback.\n" + str(e))
    gb = GradientBoostingClassifier(random_state=42)
    gb.fit(X_train, y_train)
    gb_acc = accuracy_score(y_test, gb.predict(X_test))

    print("✅ AdaBoost accuracy:        ", ada_acc)
    print("✅ GradientBoosting accuracy:", gb_acc)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.0 MB/s eta 0:00:00
✅ AdaBoost accuracy:         0.94
✅ CatBoost accuracy:         0.92
